In this notebook one can:
- load a notebook's settings as a dictionary
- change it 
- save it as a new notebook 
- submit it as a job to SLURM cluster. 

In [1]:
import sys
sys.path.append('/dls_sw/e02/software/epsic_tools')
import epsic_tools.api as ep
import pprint
import re
import subprocess
import os
from __future__ import print_function,unicode_literals
import subprocess

In [2]:
starting_notebook_path = '/dls/science/groups/e02/Mohsen/code/jupyterhub_active/SLURM_notebook_submit'
starting_notebook_name = 'test_notebook'
nb = ep.notebook_utils.NotebookHelper(starting_notebook_path, starting_notebook_name)

In [3]:
old_settings = nb.get_settings(1) # settings should be cell index 1

pprint.pprint(old_settings)

{'global_save_2D_azimuthal_projection': {'explanation': ' Set to true to save '
                                                        'the 2D r-theta '
                                                        'projectionof the full '
                                                        '4D data set - this '
                                                        'will be several GB.',
                                         'prompt': 'save r-theta projection.',
                                         'value': 'False'},
 'global_use_binned_data': {'explanation': 'Perform analysis on real space '
                                           'binned data.',
                            'prompt': 'use binned data',
                            'value': 'False'},
 'remex_hints': {'explanation': 'Hints to the workflow engine regarding '
                                'cluster processing. Please consult the '
                                'beamline staff before changing this. Valid '
   

In [4]:
# make some changes in new setting
new_setting = old_settings.copy()
new_setting['global_save_2D_azimuthal_projection'] = True
pprint.pprint(new_setting)

{'global_save_2D_azimuthal_projection': True,
 'global_use_binned_data': {'explanation': 'Perform analysis on real space '
                                           'binned data.',
                            'prompt': 'use binned data',
                            'value': 'False'},
 'remex_hints': {'explanation': 'Hints to the workflow engine regarding '
                                'cluster processing. Please consult the '
                                'beamline staff before changing this. Valid '
                                'choices are: standard_science_cluster, '
                                'ptypy_mpi, ptyrex_mpi, local',
                 'prompt': 'remote execution hints',
                 'value': 'slurm'}}


In [6]:
# Save a new version of the notebook with new settings:
save_path = '/dls/science/groups/e02/Mohsen/code/jupyterhub_active/SLURM_notebook_submit/test_2'
new_notebook_path = os.path.join(save_path, 'new_version.ipynb')
nb.set_settings(new_setting, new_notebook_path)

In [7]:
print(f'new notebook path: {new_notebook_path}')

new notebook path: /dls/science/groups/e02/Mohsen/code/jupyterhub_active/SLURM_notebook_submit/test_2/new_version.ipynb


In [8]:
# Create a bash script to submit to SLURM 
bash_script_path = os.path.join(save_path, 'cluster_submit.sh')
with open (bash_script_path, 'w') as f:
    f.write('''#!/usr/bin/env bash
#SBATCH --partition=cs04r
#SBATCH --job-name epsic_notebook
#SBATCH --time=05:00:00
#SBATCH --nodes=1
#SBATCH --tasks-per-node=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=60G
'''
f"#SBATCH --error={save_path}{os.sep}error_%j.out\n"
f"#SBATCH --output={save_path}{os.sep}output_%j.out\n"
f"module load python/epsic3.10\n"
f"jupyter nbconvert --to notebook --inplace --ClearMetadataPreprocessor.enabled=True {new_notebook_path}\n"
f"jupyter nbconvert --to notebook --allow-errors --execute {new_notebook_path}\n"
           )

In [9]:

sshProcess = subprocess.Popen(['ssh',
                               '-tt',
                               'wilson'],
                               stdin=subprocess.PIPE, 
                               stdout = subprocess.PIPE,
                               universal_newlines=True,
                               bufsize=0)
sshProcess.stdin.write("ls .\n")
sshProcess.stdin.write("echo END\n")
sshProcess.stdin.write(f"sbatch {bash_script_path}\n")
sshProcess.stdin.write("uptime\n")
sshProcess.stdin.write("logout\n")
sshProcess.stdin.close()


for line in sshProcess.stdout:
    if line == "END\n":
        break
    print(line,end="")

#to catch the lines up to logout
for line in  sshProcess.stdout: 
    print(line,end="")


 _______   __        ______         __    __  _______    ______
|       \ |  \      /      \       |  \  |  \|       \  /      \
| $$$$$$$\| $$     |  $$$$$$\      | $$  | $$| $$$$$$$\|  $$$$$$\
| $$  | $$| $$     | $$___\$$      | $$__| $$| $$__/ $$| $$   \$$
| $$  | $$| $$      \$$    \       | $$    $$| $$    $$| $$
| $$  | $$| $$      _\$$$$$$\      | $$$$$$$$| $$$$$$$ | $$   __
| $$__/ $$| $$_____|  \__| $$      | $$  | $$| $$      | $$__/  \
| $$    $$| $$     \\$$    $$      | $$  | $$| $$       \$$    $$
 \$$$$$$$  \$$$$$$$$ \$$$$$$        \$$   \$$ \$$        \$$$$$$

Welcome To Wilson - The DLS Slurm Cluster
For Help or Support - Visit: https://schelpdesk.diamond.ac.uk

Please Refrain From Running Any Tasks Directly On This Node - It Is A Submission Node Only

For jobs needing to access the GPFS02 filesystem please use the cs04r partition, for the GPFS03 filesystem please use the cs05r partition status

To get an interactive session on a GPFS02 node enter "iact02", for a GPF

Connection to wilson closed.
